In [2]:
import numpy as np
import pandas as pd
import sklearn.preprocessing as preprocessing
import tensorflow as tf


In [2]:
df = pd.read_csv(
    "datas/EURUSD_H1 copy.csv",
    sep="\t",
    # nrows=2**16,
    header=0,
    dtype={
        "open": np.float32,
        "close": np.float32,
        "tickvol": np.float32,
        "vol": np.float32,
    },
    parse_dates=[[0,1]],
    index_col=[0],
    names=["date","time","open","high","low","close","tickvol","vol","spread"])
size= len(df.index)

In [3]:
width = 8
start = len(df.index)%width
prices = np.nan_to_num(np.array(df["open"][start:]), posinf=0, neginf=0)
prices_diff = np.diff(prices[1: -64])
print(prices.shape)
# norm = tf.keras.layers.LayerNormalization(axis=1)
# nx=norm(prices).numpy()
prices_diff.std()

# print(norm(prices).numpy())

(54304,)


0.0012349293

In [44]:
print(len(df.index))
df = df.asfreq(freq='5min')
df = df.fillna(method='ffill')
print(len(df.index))

39314
54935


In [24]:
def shifted_mse(y_true, y_pred):
    y_true_sign = tf.math.softsign(y_true)
    y_pred_sign = tf.math.softsign(y_pred)
    d = tf.keras.losses.mean_squared_error(y_true + y_true_sign, y_pred + y_pred_sign)
    return d

def mse(y_true, y_pred):
    d = tf.keras.losses.mean_squared_error(y_true, y_pred)
    return d 


In [44]:
x=np.array([-1.0])
y=np.array([0.01])
print(shifted_mse(x,y).numpy())
print(mse(x,y).numpy())

2.3100990197039506
1.0201


In [3]:
norm = tf.keras.layers.LayerNormalization(axis=1)
x = np.arange(256, dtype='float32').reshape((16,16,1))
print(norm(x))


tf.Tensor(
[[[-1.6269401 ]
  [-1.4100147 ]
  [-1.1930895 ]
  [-0.9761641 ]
  [-0.7592387 ]
  [-0.54231334]
  [-0.32538807]
  [-0.10846269]
  [ 0.10846269]
  [ 0.32538807]
  [ 0.54231346]
  [ 0.75923884]
  [ 0.976164  ]
  [ 1.1930894 ]
  [ 1.4100147 ]
  [ 1.6269401 ]]

 [[-1.6269403 ]
  [-1.4100149 ]
  [-1.1930895 ]
  [-0.97616434]
  [-0.7592387 ]
  [-0.5423136 ]
  [-0.32538795]
  [-0.10846281]
  [ 0.10846233]
  [ 0.32538795]
  [ 0.5423131 ]
  [ 0.7592387 ]
  [ 0.97616386]
  [ 1.1930895 ]
  [ 1.4100146 ]
  [ 1.6269403 ]]

 [[-1.6269398 ]
  [-1.4100146 ]
  [-1.193089  ]
  [-0.97616386]
  [-0.75923824]
  [-0.5423126 ]
  [-0.32538795]
  [-0.10846233]
  [ 0.10846329]
  [ 0.32538795]
  [ 0.5423136 ]
  [ 0.7592392 ]
  [ 0.9761648 ]
  [ 1.1930895 ]
  [ 1.4100151 ]
  [ 1.6269407 ]]

 [[-1.6269407 ]
  [-1.4100151 ]
  [-1.1930895 ]
  [-0.97616386]
  [-0.7592392 ]
  [-0.5423136 ]
  [-0.32538795]
  [-0.10846233]
  [ 0.10846233]
  [ 0.32538795]
  [ 0.5423136 ]
  [ 0.75923824]
  [ 0.97616386]
  [ 1.1